In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

In [3]:
#!change the root path
root_path='/content/drive/MyDrive/nlp/project/github'

In [4]:
df_result=pd.read_csv(root_path+'/data/model_result.csv')

In [5]:
df_job_title_true=pd.read_csv(root_path+'/data/job_title_true.csv')

In [6]:
df_level_true=pd.read_csv(root_path+'/data/level_true.csv')

In [7]:
df_other_results=pd.read_csv(root_path+'/data/yargy_and_reg_ex_result.csv')

In [9]:
df=pd.merge(df_result,df_job_title_true,how='inner',left_on='Id',right_on='Id')
df=pd.merge(df,df_level_true,how='inner',left_on='Id',right_on='Id')
df=pd.merge(df,df_other_results,how='inner',left_on='Id',right_on='Id')

In [10]:
df.shape

(500, 89)

In [11]:
#delete quatas
import ast
df['model_result'] = df['model_result'].apply(lambda x: ast.literal_eval(x))
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x))

In [12]:
def list_extract_f (list_of_lists):
  return [item for i in list_of_lists for item in i]

In [13]:
df['tags1']=df['tags'].apply(lambda x:list_extract_f(x))
df['model_result1']= df['model_result'].apply(lambda x:list_extract_f(x[1]))

In [15]:
from deeppavlov.metrics import fmeasure

In [16]:
y_pred_test1=list_extract_f(df[df['data_grp']=='test']['tags1'].values.tolist())
y_true_test1=list_extract_f(df[df['data_grp']=='test']['model_result1'].values.tolist())

y_pred_test2=df[df['data_grp']=='test']['tags1'].values.tolist()
y_true_test2=df[df['data_grp']=='test']['model_result1'].values.tolist()

In [17]:
print('test_ner_f1=',fmeasure.ner_f1(y_true_test2, y_pred_test2))
print('test_ner_token_f1=',fmeasure.ner_token_f1(y_true_test2, y_pred_test2))
fmeasure.precision_recall_f1(y_true_test1, y_pred_test1)

test_ner_f1= 69.12280701754386
test_ner_token_f1= 80.3500397772474


OrderedDict([('LOC',
              OrderedDict([('precision', 72.5),
                           ('recall', 69.04761904761905),
                           ('f1', 70.73170731707319),
                           ('n_pred', 40),
                           ('n_true', 42),
                           ('tp', 29),
                           ('fn', 13),
                           ('fp', 11)])),
             ('SAL',
              OrderedDict([('precision', 81.81818181818183),
                           ('recall', 63.1578947368421),
                           ('f1', 71.2871287128713),
                           ('n_pred', 44),
                           ('n_true', 57),
                           ('tp', 36),
                           ('fn', 21),
                           ('fp', 8)])),
             ('VAC',
              OrderedDict([('precision', 76.28205128205127),
                           ('recall', 63.63636363636363),
                           ('f1', 69.38775510204081),
                      

In [18]:
#tags extraction
def trunc_tags_f(tags_list):
  for i, lst in enumerate(tags_list):
    for j, tag in enumerate(lst):
      if len(tag) >1:
        tags_list[i][j]=tag[2:]

for i, item in enumerate(df['model_result']):
  trunc_tags_f(item[1])

def find_tags_in_result_f (tags_list, tag):
  index_list=[]
  for k, item in enumerate(tags_list[1]):
    i=0
    while True:
      try:
        j=item[i:].index(tag)+i
        index_list.append([k,j])    
        i=j+1
      except:
        break
  word_list=[]
  for i in index_list:
    word_list.append(tags_list[0][i[0]][i[1]])
  return word_list

df['Vacancy_res']= np.empty((len(df), 0)).tolist()
df['Salary_res']= np.empty((len(df), 0)).tolist()
df['Location_res']= np.empty((len(df), 0)).tolist()
df['WorkStyle_res']= np.empty((len(df), 0)).tolist()

for i, item in enumerate(df['model_result']):
  df.at[i,'Vacancy_res']=find_tags_in_result_f (item,'VAC')
  df.at[i,'Salary_res']=find_tags_in_result_f (item,'SAL')
  df.at[i,'Location_res']=find_tags_in_result_f (item,'LOC')
  df.at[i,'WorkStyle_res']=find_tags_in_result_f (item,'WST')

In [19]:
#yargy_parser result preprocessing
title_col=['title1','title2','title3','title4','title5','title6','title7','title8','title9','title10','title11','title12','title13','title14','title15','title16','title17']
grade_col=['grade1','grade2','grade3','grade4','grade5','grade6','grade7','grade8','grade9','grade10','grade11','grade12','grade13','grade14','grade15','grade16','grade17','grade18']
salary_col=['vilka1','vilka2','vilka3','vilka4','vilka5','vilka6']
location_col=['city1', 'city2', 'city3', 'city4','city5', 'city6', 'city7',  'city8', 'city9', 'metro1', 'metro2', 'metro3', 'metro4']

df['da_flg_yar'] =0
df['ds_flg_yar'] =0
df['de_flg_yar'] =0
df['vac_any_flg_yar'] =0

df.loc[df[title_col].eq('DA').any(1),'da_flg_yar']=1
df.loc[df[title_col].eq('DS').any(1),'ds_flg_yar']=1
df.loc[df[title_col].eq('DE').any(1),'de_flg_yar']=1
df.loc[df[title_col].notnull().any(1),'vac_any_flg_yar']=1

df['junior_flg_yar'] =0
df['middle_flg_yar'] =0
df['senior_flg_yar'] =0
df['lead_flg_yar'] =0
df['vac_level_any_flg_yar'] =0

df.loc[df[grade_col].eq('intern').any(1),'junior_flg_yar']=1
df.loc[df[grade_col].eq('junior').any(1),'junior_flg_yar']=1
df.loc[df[grade_col].eq('middle').any(1),'middle_flg_yar']=1
df.loc[df[grade_col].eq('senior').any(1),'senior_flg_yar']=1
df.loc[df[grade_col].eq('lead').any(1),'lead_flg_yar']=1
df.loc[df[grade_col].notnull().any(1),'vac_level_any_flg_yar']=1

df['salary_any_flg_yar'] =0
df.loc[df[grade_col].notnull().any(1),'salary_any_flg_yar']=1

df['location_any_flg_yar'] =0
df.loc[df[location_col].notnull().any(1),'location_any_flg_yar']=1


/usr/local/lib/python3.7/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [20]:
#reg_expr result preprocessing
df['forks'] = df['forks'].apply(lambda x: ast.literal_eval(x))
df['salary_reg_expr_flg']=0
for i,item in enumerate(df['salary_reg_expr_flg']):
  df.at[i,'salary_reg_expr_flg']= 1 if len(df.loc[i,['forks']][0])>0 else 0

In [21]:
#true result preprocessing
df['Vacancy_true_any']=0
for i,item in enumerate(df['Vacancy_true_any']):
 df.at[i,'Vacancy_true_any']= 1 if df.loc[i,['da_true','ds_true','de_true','other_true']].sum()>0 else 0

df['vac_level_true_any']=0
for i,item in enumerate(df['vac_level_true_any']):
 df.at[i,'vac_level_true_any']= 1 if df.loc[i,['junior_true','middle_true','senior_true','lead_true']].sum()>0 else 0

In [22]:
# model result extraction with regular expression
import re

da_ex = '(' + '|'.join(['analyst', 'аналит', 'биайщик', '\Wbi\W', '\Wda\W','Analytic']) + ')'
ds_ex = '(' + '|'.join(['scientist', 'сай?е?нтист', 'сатанист', 'исследовател', 'researcher', '\Wds\W', '\Wml\W'\
                        , 'science','Computer.Vision','CV.Engineer','ML.Engineer','Research.Engineer','DL.developer','CV.developer','AI.Engineer'
                        ,'DL.researcher','CV.researcher','распознавание.речи','ML.Инженер'
                        , '\Wds\W', '\Wдс\W', '\Wds\W', '\Wдс\W','NLP.Engineer','Machine.Learning','\WCV\W','\WNLP\W','\WMLE\W','Deep.Learning','search.engineer','машин.*обуч'
                        ,'recommender.system','компьютерному.зрен','\WDL\W','ai.specialist','reinforcement.*learning']) + ')'
de_ex = '(' + '|'.join(['Data.engineer|architect|архитектор','Data.Инженер','\Wde\W','\Wде\W','D.engineer','инженер.*данн','ETL','Hadoop','DataEngineer']) + ')'
other_ex = '(' + '|'.join(['product.manager','разработчиков.Qlik.Sense','разработчик.Python','владел','software.engineer','iOS','проект','код.ревьюер','DevOps','Python.*Developer']) + ')'

df['Vacancy_res_da']=0
df['Vacancy_res_de']=0
df['Vacancy_res_ds']=0
df['Vacancy_res_other']=0

for i,item in enumerate(df['Vacancy_res']):
 df.at[i,'Vacancy_res_da']=1 if len(re.findall(da_ex, ' '+' '.join(item)+' ', re.IGNORECASE))>0 else 0
 df.at[i,'Vacancy_res_ds']=1 if len(re.findall(ds_ex, ' '+' '.join(item)+' ', re.IGNORECASE))>0 else 0
 df.at[i,'Vacancy_res_de']=1 if len(re.findall(de_ex, ' '+' '.join(item)+' ', re.IGNORECASE))>0 else 0
 df.at[i,'Vacancy_res_other']=1 if len(re.findall(other_ex, ' '+' '.join(item)+' ', re.IGNORECASE))>0 else 0

df['Vacancy_res_any']=0
for i,item in enumerate(df['Vacancy_res']):
 df.at[i,'Vacancy_res_any']=1 if df.loc[i,['Vacancy_res_da','Vacancy_res_ds','Vacancy_res_de','Vacancy_res_other']].sum()>0 else 0

levels_dict = {
    'junior': r'(jun|джун|начинающ|стаж[её]р)',
    'middle': r'(mid|\Wмидл|миддл)',
    'senior': r'(senior|синьор|сеньор|\Wстарш)',
    'lead': r'(\Wlead\W|\Whead\W|\Wлид\W|\Wведущ|Chief|руковод)',
}

df['Vacancy_res_junior']=0
df['Vacancy_res_middle']=0
df['Vacancy_res_senior']=0
df['Vacancy_res_lead']=0

for grade in levels_dict.keys():
  for i,item in enumerate(df['Vacancy_res']):
    #df.at[i,'Vacancy_res_'+grade]=str(re.findall(levels_dict[grade], ' '+' '.join(item)+' ', re.IGNORECASE))
    df.at[i,'Vacancy_res_'+grade]=1 if len(re.findall(levels_dict[grade], ' '+' '.join(item)+' ', re.IGNORECASE))>0 else 0

df['Salary_res_flg']=0
df['Location_res_flg']=0
for i,item in enumerate(df['Salary_res']):
 df.at[i,'Salary_res_flg']=1 if len( df.at[i,'Salary_res'])>0 else 0
 df.at[i,'Location_res_flg']=1 if len( df.at[i,'Location_res'])>0 else 0

In [23]:
from sklearn.metrics import f1_score

In [24]:
#job title metrics
l_true=df[df['data_grp']=='test'][['da_true','ds_true','de_true']].values
l_yargy=df[df['data_grp']=='test'][['da_flg_yar','ds_flg_yar','de_flg_yar']].values
l_deep=df[df['data_grp']=='test'][['Vacancy_res_da','Vacancy_res_ds','Vacancy_res_de']].values
l_regexpr=df[df['data_grp']=='test'][['Data Analyst', 'Data Scientist', 'Data Engineer']].values
print('f1_ner=',f1_score(l_true, l_deep, average=None))
print('f1_yargy=',f1_score(l_true, l_yargy, average=None))
print('f1_reg_expr=',f1_score(l_true, l_regexpr, average=None))

print('f1_ner_total=',f1_score(l_true, l_deep, average='micro'))
print('f1_yargy_total=',f1_score(l_true, l_yargy, average='micro'))
print('f1_reg_expr_total=',f1_score(l_true, l_regexpr, average='micro'))

l_true=df[df['data_grp']=='test'][['Vacancy_true_any']].values
l_yargy=df[df['data_grp']=='test'][['vac_any_flg_yar']].values
l_deep=df[df['data_grp']=='test'][['Vacancy_res_any']].values
print('f1_ner_any=',f1_score(l_true, l_deep))
print('f1_yargy_any=',f1_score(l_true, l_yargy))


f1_ner= [0.88888889 0.93670886 0.93333333]
f1_yargy= [0.75       0.72463768 0.7       ]
f1_reg_expr= [0.57142857 0.90909091 0.875     ]
f1_ner_total= 0.9320388349514563
f1_yargy_total= 0.7216494845360826
f1_reg_expr_total= 0.8799999999999999
f1_ner_any= 0.9690721649484536
f1_yargy_any= 0.9010989010989011


In [25]:
#job level metrics
l_true=df[df['data_grp']=='test'][['junior_true','middle_true','senior_true','lead_true']].values
l_yargy=df[df['data_grp']=='test'][['junior_flg_yar','middle_flg_yar','senior_flg_yar','lead_flg_yar']].values
l_deep=df[df['data_grp']=='test'][['Vacancy_res_junior','Vacancy_res_middle','Vacancy_res_senior','Vacancy_res_lead']].values
l_regexpr=df[df['data_grp']=='test'][['Junior', 'Middle', 'Senior','Lead']].values
print('f1_ner=',f1_score(l_true, l_deep, average=None))
print('f1_yargy=',f1_score(l_true, l_yargy, average=None))
print('f1_reg_expr=',f1_score(l_true, l_regexpr, average=None))

print('f1_ner_total=',f1_score(l_true, l_deep, average='micro'))
print('f1_yargy_total=',f1_score(l_true, l_yargy, average='micro'))
print('f1_reg_expr_total=',f1_score(l_true, l_regexpr, average='micro'))


f1_ner= [0.875 0.95  0.95  1.   ]
f1_yargy= [0.88888889 0.97560976 0.92682927 0.71428571]
f1_reg_expr= [0.94736842 0.95       0.93023256 0.76923077]
f1_ner_total= 0.9433962264150945
f1_yargy_total= 0.912280701754386
f1_reg_expr_total= 0.9217391304347826


In [26]:
#salary metrics
df[['data_grp','Salary_res_flg','salary_any_flg_yar','salary_reg_expr_flg']].groupby(by=["data_grp"]).sum()/df[['data_grp','Salary_res_flg','salary_any_flg_yar','salary_reg_expr_flg']].groupby(by=["data_grp"]).count()

,Salary_res_flg,salary_any_flg_yar,salary_reg_expr_flg
data_grp,,,
test,0.840000,0.720000,1.000000
train,0.920398,0.676617,0.945274
valid,0.875000,0.687500,0.958333


In [27]:
#location metrics
df[['data_grp','Location_res_flg','location_any_flg_yar']].groupby(by=["data_grp"]).sum()/df[['data_grp','Location_res_flg','location_any_flg_yar']].groupby(by=["data_grp"]).count()

,Location_res_flg,location_any_flg_yar
data_grp,,
test,0.600000,0.720000
train,0.791045,0.771144
valid,0.708333,0.687500
